In [17]:
from __future__ import division, print_function, unicode_literals
import csv
import shutil
import os
import time
import requests
import subprocess
from subprocess import call
import numpy as np
import tensorflow as tf
from rl.callbacks import FileLogger, ModelIntervalCheckpoint
import pandas as pd 


from keras import backend as K



import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, GRU
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory 
from keras.callbacks import TensorBoard

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

Using TensorFlow backend.


#   MDP 


In [18]:
from subprocess import Popen, PIPE
import os
from tempfile import mkdtemp
from werkzeug import secure_filename
import requests
import gym
from gym import error, spaces, utils
from gym.utils import seeding
from gym import spaces, logger
import subprocess
from subprocess import Popen, PIPE
import numpy as np 
import time 
import csv
import pandas as pd 

class MicroEnv(gym.Env):
    metadata = {'render.modes': ['human']}
    def __init__(self):
        print('__init__')
        tstart = time.time()
        #filename1="dqn_logsv4.csv"
        #self.fileHandle = open(filename1,"w")
        #self.writer = csv.writer(self.fileHandle)
        self.maxNode = 2 
        self.minNode = 1 
        self.node = 1
        self.ip = '192.168.99.100'
        self.cluster= False
        #self.ip = subprocess.check_output(["docker-machine", "ip", "new-mgr"])
        self.cpu_axis  = self.get_cpu_observation()
        self.mem_axis = self.get_mem_observation()
        self.disk_axis = self.get_disk_observation()
        self.net_axis  = self.get_net_observation()
        self.action_space = spaces.Discrete(10)
        self.action_space = spaces.Box(0, 10, shape=(1, 1), dtype=np.int16)
        high = np.array([
            self.get_cpu_observation(),
            self.get_mem_observation(),
            self.get_disk_observation(),
            self.get_net_observation()])
        low = np.array([
            np.zeros(5),
            np.zeros(5),
            np.zeros(5),
            np.zeros(5)])
        self.observation_space = spaces.Box(low, high, dtype=np.float32)
        self.seed()
        self.obs= 0
        self.obs = self.get_observation()
        self.viewer = None
        self.state = self.get_observation()
        self.state_name = 's0'
        self.attempt = 0 
        self.steps_beyond_done = None
        self.done = False
        self.adapte_cpu= False  
        self.adapte_mem= False 
        self.adapte_disk= False 
        self.adapte_net= False 
        #self.writer.writerow(["timestamp","state", "action", "reward", "maxUtility", "duration", "info", "# node"])
    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]
    
    def step(self, action):
        #assert self.action_space.contains(action), "%r (%s) invalid"%(action, type(action))
        action = np.floor(action)
        state = self.get_observation()
        #past_stat = self.state
        #print(state)
        #Find the Utility Prefernces 
        # Select action 
        # get the reward value
        tstart = time.time()
         
        self.attempt += 1
        #print('self.adapte_cpu: ', self.adapte_cpu, 'self.adapte_mem:', self.adapte_mem, 'self.adapte_disk:',self.adapte_disk,'self.adapte_net:', self.adapte_net )
        done=False
        reward=0
        info=''
        print ("Starting Action: ", action)
        if action==0:
            #print("Stay in State S0")
            self.obs = self.get_observation()

            done = False
            self.state_name='s0'
        elif action == 1:
            #print("Stay in State S1")
            self.obs = self.get_observation()
            done = True
            self.state_name='s1'

        elif action == 2:
            self.obs = self.get_observation()
            try:
                response = requests.get('http://'+self.ip+':8888/services/vscale/web/'+ str(self.attempt) + '/' + str(self.cpu_axis[0])+'/'+str(self.cpu_axis[3]))
                results = response.json()
                if results['result']=='Service converged':
                    done=True 
                    self.obs = self.get_observation()
                    
                    self.state_name='s2'
                    info = "Scale Up Move to State S2"
                else:
                    done= False
                    print(results)
                    self.obs = self.get_observation()
                    
                     
                    self.state_name='s0'
                    info = "S2 => No Scale Move back to State S0"
            except:
                pass
            finally:
                
                pass

        elif action == 3: 
            #print("Maintain Cluster State S4 and delete dangling docker containers")
            self.obs = self.get_observation()
             
            #print("reward: ",reward, np.amax(self.obs[:,4]))
            done= False
            self.state_name='s0'
            info = "delete dangling docker containers S3"
            cur_dir = os.getcwd()
            filepath = os.path.join(cur_dir, 'cleancontainers.sh')
            print (filepath)
            res= subprocess.call(filepath, shell=True)
            #print (res)
             
            self.state_name='s3'
            done = True
        elif action == 4:
            self.obs = self.get_observation()
            if (self.node <= self.maxNode):
                #print("Add Node s4")
                cur_dir = os.getcwd()
                filepath = os.path.join(cur_dir, 'addNode.sh')
                print (filepath)
                res= subprocess.call(filepath, shell=True)
                print (res)
                info = "Add Node S4"
                self.obs = self.get_observation()
                 
                self.state_name='s4'
                
                done= True
                self.node +=1
            elif self.node==5:
                    done=True 
                    
                    self.state_name='s4'
            else:
                #print("go back to Cluster State at S0: ", self.attempt)
                done= False
                self.obs = self.get_observation()
                 
                self.state_name='s0'
        elif action == 5:
            self.obs = self.get_observation()
            if (self.node <= self.maxNode and self.node > self.minNode ):
                #print("Delete Node S5")
                self.state_name='s5'
                cur_dir = os.getcwd()
                filepath = os.path.join(cur_dir, 'deleteNode.sh')
                #print (filepath)
                res= subprocess.call(filepath, shell=True)
                #print (res)
                
                info = "Delete Node S5"
                 
                #print("reward: ",reward, np.amax(self.obs[:,4]))
                done= True
                self.node -=1
            else:
                #print("Maintain Cluster State at S0: ", self.attempt, self.node, self.minNode, self.maxNode)
                done= False
                 
                #print("reward: ",reward)
                self.state_name='s0'
                if self.node==1:
                    done=True 
                     
                    self.state_name='s5'
        elif action==6:
            self.obs = self.get_observation()
            #print("freedisk Space S6")
            cur_dir = os.getcwd()
            filepath = os.path.join(cur_dir, 'freedisk.sh')
            #print (filepath)
            res= subprocess.call(filepath, shell=True)
            #print (res)
            info = "freedisk Node S6"
            self.state_name='s6'
             
            #print("reward: ",reward, np.amax(self.obs[:,4]))
            done= True
        
        elif action == 7:
            self.obs = self.get_observation()
            if (self.node <= self.maxNode and self.node >= self.minNode ):
                self.state_name='s7'
                cur_dir = os.getcwd()
                filepath = os.path.join(cur_dir, 'promoteNode.sh')
                #print (filepath)
                res= subprocess.call(filepath, shell=True)
                #print (res)
                info = "Promote Worker node to Manager S7"
                 
                #print("reward: ",reward, np.amax(self.obs[:,4]), "Node: ", self.node)
                done= True
            elif (self.node >= self.maxNode):
                info = "Maintain Manager nodes S0"
                self.state_name='s0'
                 
                #print("reward: ",reward, np.amax(self.obs[:,4]), "Node: ", self.node)
                done= True 
        elif action==8:
            self.obs = self.get_observation()
            if (self.node <= self.maxNode):
                self.state_name='s8'
                cur_dir = os.getcwd()
                filepath = os.path.join(cur_dir, 'manager.sh')
                #print (filepath)
                res= subprocess.call(filepath, shell=True)
                #print (res)
                info = "Add Manager node S8"
                self.node += 1
                 
                #print("reward: ",reward, np.amax(self.obs[:,4]), "Node: ", self.node)
                done= True
            elif (self.node >= self.maxNode):
                info = "Maintain Manager nodes S0"
                self.state_name='s8'
                 
                #print("reward: ",reward, np.amax(self.obs[:,4]), "Node: ", self.node)
                done= True 
            else:
                #print("Maintain Cluster State S0", self.attempt, self.node )
                self.state_name='s0'
                 
                done= False
                info = "from S8 to S0 "
                
        elif action == 9:
            self.obs = self.get_observation()
            if not self.cluster:
                #add new cluster one time only
                try:         
                    print("roll-back and enforce new cluster")
                    cur_dir = os.getcwd()
                    filepath = os.path.join(cur_dir, 'nupicnewcluster.sh')
                    print (filepath)
                    res= subprocess.call(filepath, shell=True)
                     
                    self.state_name='s9'
                    info = "rollback and enforce new  cluster S9"
                    #time.sleep(300)
                    myip = subprocess.check_output(["docker-machine", "ip", "nupic"])
                    self.ip=myip.decode('utf-8').strip()
                     
                    #print("reward: ",reward, np.amax(self.obs[:,4]))
                    done= True
                    self.cluster = True
                    #print(reward)
                    info = "roll-back and enforce new cluster"
                except:
                    pass
                finally:
                    pass
            elif self.cluster:
                self.state_name='s9'
                 
                done= False
                info = "from S9 to S0 "
                 
                     

            
        else: 
            print ("action not defined")
            self.state_name='s0'
            self.obs = self.get_observation()
            done= False
            reward= -1  
            info = "action not defined"
        if done:
            reward = 1 
        else:
            reward= 1 - np.amax(self.obs[3]) 
            
        tend = tstart - time.time()
        print ("\nState: ", self.state_name, "action: ", action, "reward: ", reward, "# nodes:", self.node, "cluster: ", self.cluster )
        #writer1.writerow(["timestamp","state", "action", "reward", "maxUtility", "duration"])
        self.obs = self.get_observation()
        #self.writer.writerow([tstart,self.state_name, action, reward, np.amax(self.obs[:,4]), tend, info, self.node])
        return self.obs, reward, done, {}
    def reset(self):
        #self.fileHandle.close()
        self.state = self.get_observation()
        self.steps_beyond_done = None
        self.adapte_cpu= False  
        self.adapte_mem= False 
        self.adapte_disk= False 
        self.adapte_net= False
        cur_dir = os.getcwd()
        filepath = os.path.join(cur_dir, 'deleteallnodes.sh')
        
        print (filepath)
        #res= subprocess.call(filepath, shell=True)
        self.maxNode = 3 
        self.minNode = 1 
        #self.node = 1
        #self.cluster= False
        return self.state
    def render(self, mode='human', close=False):
        logger.warn("View is not allowed in this environment")
        return 0 
    def close(self):
        self.fileHandle.close()
        if self.viewer:
            self.viewer.close()
            self.viewer = None
    def get_observation(self):
        try:
            self.disk_axis = self.get_disk_observation() 
            self.mem_axis = self.get_mem_observation()
            self.cpu_axis = self.get_cpu_observation()
            self.net_axis = self.get_net_observation()
            obs = np.vstack((self.cpu_axis,self.mem_axis, self.disk_axis, self.net_axis) )
            obs2 = obs.reshape(1,20)
            return np.squeeze(obs2)
        except:
            pass
        finally: 
            pass 
    def get_cpu_observation(self):
        try: 
            response = requests.get('http://'+self.ip+':8888/cpu', timeout=5)
            results = response.json()
            if len(results) > 0:
                    cpu = results['cpu']
                    prediction = results['prediction']
                    anomalyScore = results['anomalyScore']
                    anomalyLikelihood = results['anomalyLikelihood']
                    utility_cpu = results['utility_cpu']
                    cpu_axis=[cpu, prediction, anomalyScore, anomalyLikelihood, utility_cpu]
        except:
            cpu_axis=[-1, -1, -1, -1, -1]
            pass    
        finally: 
            pass
        return np.array(cpu_axis)
    def get_mem_observation(self):
        try:
            response = requests.get('http://'+self.ip+':8888/mem', timeout=5)
            results = response.json()
            if len(results) > 0:
                mem = results['mem']
                prediction = results['prediction']
                anomalyScore = results['anomalyScore']
                anomalyLikelihood = results['anomalyLikelihood']
                utility_mem = results['utility_mem']
                mem_axis=[mem, prediction, anomalyScore, anomalyLikelihood, utility_mem]
        except:
            mem_axis=[-1, -1, -1, -1, -1]
            pass     
        finally: 
            #mem_axis=[0, 0, 0, 0, 0]
            pass
        return np.array(mem_axis)
    def get_net_observation(self):
        try:
            response = requests.get('http://'+self.ip+':8888/net', timeout=5)
            results = response.json()
            if len(results) > 0:
                net = results['net']
                prediction = results['prediction']
                anomalyScore = results['anomalyScore']
                anomalyLikelihood = results['anomalyLikelihood']
                utility_net = results['utility_net']
                net_axis=[net, prediction, anomalyScore, anomalyLikelihood, utility_net]
        except:
            net_axis=[-1, -1, -1, -1, -1]
            pass
        finally:
            pass 
        return np.array(net_axis)
    def get_disk_observation(self):
        try:
            response = requests.get('http://'+self.ip+':8888/disk', timeout=5)
            if response is not None:
                results = response.json()
                if len(results) > 0:
                    disk = results['disk']
                    prediction = results['prediction']
                    anomalyScore = results['anomalyScore']
                    anomalyLikelihood = results['anomalyLikelihood']
                    utility_disk = results['utility_disk']
            disk_axis=[disk, prediction, anomalyScore, anomalyLikelihood, utility_disk]
        except:
            disk_axis=[-1, -1, -1, -1, -1]
            pass
        finally:
            
            pass
        return np.array(disk_axis)

    def get_current_state(self):
        current_state = self.state
         
        return current_state
     

## Deep Q-Learning


In [19]:
import gym
#import gym_foo
import gym_microagent
#env = gym.make('foo-v16')
env =  MicroEnv()

__init__


In [20]:
env.observation_space.shape

(4, 5)

In [21]:
 env.obs

array([1.95000000e+01, 1.95000000e+01, 1.00000000e+00, 5.00000000e-01,
       9.75000000e+00, 2.86123215e+01, 2.86123215e+01, 1.00000000e+00,
       5.00000000e-01, 1.43061607e+01, 9.80000000e+01, 9.80000000e+01,
       1.00000000e+00, 5.00000000e-01, 4.90000000e+01, 3.27604826e+03,
       3.27604826e+03, 1.00000000e+00, 5.00000000e-01, 1.63802413e+03])

In [22]:
nb_actions = 10

In [23]:
np.amin(env.obs[3])

0.5

In [24]:
env.node

1

In [25]:
env.attempt

0

In [26]:
env.step(11)

Starting Action:  11.0
action not defined

State:  s0 action:  11.0 reward:  0.5 # nodes: 1 cluster:  False


(array([2.99000000e+01, 2.99000000e+01, 1.00000000e+00, 5.00000000e-01,
        1.49500000e+01, 2.85648603e+01, 2.85648603e+01, 1.00000000e+00,
        5.00000000e-01, 1.42824301e+01, 9.80000000e+01, 9.80000000e+01,
        1.00000000e+00, 5.00000000e-01, 4.90000000e+01, 2.52943137e+03,
        2.52943137e+03, 1.00000000e+00, 5.00000000e-01, 1.26471569e+03]),
 0.5,
 False,
 {})

In [27]:
# Test model in CartPole 

ENV_NAME = 'CartPole-v0'


# Get the environment and extract the number of actions.
env2 = gym.make(ENV_NAME)
np.random.seed(123)
env2.seed(123)
#nb_actions = env.action_space.n
#nb_actions

env2.observation_space.shape

In [28]:
obs = env.reset()


/Users/baz/ieee-demo/deleteallnodes.sh


In [29]:
env.step(11)

Starting Action:  11.0
action not defined

State:  s0 action:  11.0 reward:  0.5 # nodes: 1 cluster:  False


(array([29.9       , 29.9       ,  1.        ,  0.5       , 14.95      ,
        28.56486028, 28.56486028,  1.        ,  0.5       , 14.28243014,
        98.        , 98.        ,  1.        ,  0.5       , 49.        ,
         0.        ,  0.        ,  1.        ,  0.5       ,  0.        ]),
 0.5,
 False,
 {})

In [30]:
ob = env.get_observation()

In [31]:
ob.shape

(20,)

In [32]:
np.squeeze(ob).shape

(20,)

In [42]:
actorrnn = Sequential()
actorrnn.add(GRU(units=20,
              return_sequences=False,
              input_shape=  (1,)+ obs.shape))
actorrnn.add(Dense(16, activation='relu'))

 
actorrnn.add(Dense(16, activation='relu'))


 
actorrnn.add(Dense(16, activation='relu'))


 
actorrnn.add(Dense(nb_actions, activation='linear'))


actorrnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_5 (GRU)                  (None, 20)                2460      
_________________________________________________________________
dense_9 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_10 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_11 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_12 (Dense)             (None, 10)                170       
Total params: 3,510
Trainable params: 3,510
Non-trainable params: 0
_________________________________________________________________


model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

In [44]:
from keras.utils import plot_model
plot_model(actorrnn, show_shapes=True, expand_nested=False, rankdir='TB', to_file='drqn.png')
 

In [41]:
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=actorrnn, nb_actions=nb_actions, memory=memory, nb_steps_warmup=100,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

ValueError: Model output "Tensor("activation_2/Identity:0", shape=(1, 10), dtype=float32)" has invalid shape. DQN expects a model that has one dimension for each action, in this case 10.

In [20]:

ENV_NAME="microv2"
weights_filename = 'DQRN_{}_weights.h5f'.format(ENV_NAME)
checkpoint_weights_filename = 'dqn_' + ENV_NAME + '_weights_{step}.h5f'
 
log_filename = 'DQRN_{}_log.json'.format(ENV_NAME)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]
log_dir = './DQRN_{}_log/'.format(ENV_NAME)
callbacks += [TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=False)]
#dqn.fit(env, callbacks=callbacks, nb_steps=5000, log_interval=10000,verbose=1)
#dqn.fit(env, callbacks=callbacks, nb_steps=5000, visualize=False, log_interval=10000, verbose=2)
dqn.fit(env, nb_steps=5000, callbacks=callbacks, visualize=False, verbose=1, nb_max_episode_steps=500, log_interval=10000)

Training for 5000 steps ...
/Users/baz/ieee-demo/deleteallnodes.sh
Interval 1 (0 steps performed)
Starting Action:  5.0

State:  s5 action:  5.0 reward:  1 # nodes: 1 cluster:  False
    1/10000 [..............................] - ETA: 4:33:44 - reward: 1.0000/Users/baz/ieee-demo/deleteallnodes.sh
Starting Action:  9.0
roll-back and enforce new cluster
/Users/baz/ieee-demo/nupicnewcluster.sh

State:  s9 action:  9.0 reward:  1 # nodes: 1 cluster:  True
    2/10000 [..............................] - ETA: 59:02:26 - reward: 1.0000/Users/baz/ieee-demo/deleteallnodes.sh
Starting Action:  5.0

State:  s5 action:  5.0 reward:  1 # nodes: 1 cluster:  True
    3/10000 [..............................] - ETA: 39:24:45 - reward: 1.0000/Users/baz/ieee-demo/deleteallnodes.sh
Starting Action:  5.0

State:  s5 action:  5.0 reward:  1 # nodes: 1 cluster:  True
    4/10000 [..............................] - ETA: 29:35:33 - reward: 1.0000/Users/baz/ieee-demo/deleteallnodes.sh
Starting Action:  2.0

State

   93/10000 [..............................] - ETA: 15:05:35 - reward: 1.4839Starting Action:  2.0

State:  s9 action:  2.0 reward:  2 # nodes: 4 cluster:  True
   94/10000 [..............................] - ETA: 14:55:59 - reward: 1.4894Starting Action:  2.0

State:  s9 action:  2.0 reward:  2 # nodes: 4 cluster:  True
   95/10000 [..............................] - ETA: 14:46:35 - reward: 1.4947Starting Action:  4.0

State:  s0 action:  4.0 reward:  2 # nodes: 4 cluster:  True
   96/10000 [..............................] - ETA: 14:37:24 - reward: 1.5000Starting Action:  5.0

State:  s0 action:  5.0 reward:  2 # nodes: 4 cluster:  True
   97/10000 [..............................] - ETA: 14:28:22 - reward: 1.5052Starting Action:  9.0

State:  s9 action:  9.0 reward:  2 # nodes: 4 cluster:  True
   98/10000 [..............................] - ETA: 14:19:32 - reward: 1.5102Starting Action:  2.0

State:  s9 action:  2.0 reward:  2 # nodes: 4 cluster:  True
   99/10000 [.....................

  138/10000 [..............................] - ETA: 10:27:01 - reward: 1.5072Starting Action:  3.0
/Users/baz/ieee-demo/cleancontainers.sh

State:  s3 action:  3.0 reward:  1 # nodes: 4 cluster:  True
  139/10000 [..............................] - ETA: 10:22:58 - reward: 1.5036/Users/baz/ieee-demo/deleteallnodes.sh
Starting Action:  2.0

State:  s3 action:  2.0 reward:  2 # nodes: 4 cluster:  True
  140/10000 [..............................] - ETA: 10:18:34 - reward: 1.5071Starting Action:  3.0
/Users/baz/ieee-demo/cleancontainers.sh

State:  s3 action:  3.0 reward:  1 # nodes: 4 cluster:  True
  141/10000 [..............................] - ETA: 10:14:39 - reward: 1.5035/Users/baz/ieee-demo/deleteallnodes.sh
Starting Action:  0.0

State:  s0 action:  0.0 reward:  2 # nodes: 4 cluster:  True
  142/10000 [..............................] - ETA: 10:10:21 - reward: 1.5070Starting Action:  8.0

State:  s8 action:  8.0 reward:  1 # nodes: 4 cluster:  True
  143/10000 [........................

  183/10000 [..............................] - ETA: 7:59:30 - reward: 1.5027Starting Action:  5.0

State:  s0 action:  5.0 reward:  2 # nodes: 4 cluster:  True
Starting Action:  5.0

State:  s0 action:  5.0 reward:  2 # nodes: 4 cluster:  True
  185/10000 [..............................] - ETA: 7:54:19 - reward: 1.5081Starting Action:  7.0

State:  s0 action:  7.0 reward:  1 # nodes: 4 cluster:  True
  186/10000 [..............................] - ETA: 7:51:46 - reward: 1.5054/Users/baz/ieee-demo/deleteallnodes.sh
Starting Action:  4.0

State:  s0 action:  4.0 reward:  2 # nodes: 4 cluster:  True
  187/10000 [..............................] - ETA: 7:49:17 - reward: 1.5080Starting Action:  4.0

State:  s0 action:  4.0 reward:  2 # nodes: 4 cluster:  True
  188/10000 [..............................] - ETA: 7:46:48 - reward: 1.5106Starting Action:  3.0
/Users/baz/ieee-demo/cleancontainers.sh

State:  s3 action:  3.0 reward:  1 # nodes: 4 cluster:  True
  189/10000 [........................

  282/10000 [..............................] - ETA: 5:15:56 - reward: 1.6064Starting Action:  5.0

State:  s0 action:  5.0 reward:  2 # nodes: 4 cluster:  True
  283/10000 [..............................] - ETA: 5:14:51 - reward: 1.6078Starting Action:  0.0

State:  s0 action:  0.0 reward:  2 # nodes: 4 cluster:  True
  284/10000 [..............................] - ETA: 5:13:46 - reward: 1.6092Starting Action:  5.0

State:  s0 action:  5.0 reward:  2 # nodes: 4 cluster:  True
  285/10000 [..............................] - ETA: 5:12:41 - reward: 1.6105Starting Action:  0.0

State:  s0 action:  0.0 reward:  2 # nodes: 4 cluster:  True
  286/10000 [..............................] - ETA: 5:11:36 - reward: 1.6119Starting Action:  5.0

State:  s0 action:  5.0 reward:  2 # nodes: 4 cluster:  True
  287/10000 [..............................] - ETA: 5:10:33 - reward: 1.6132Starting Action:  2.0

State:  s0 action:  2.0 reward:  2 # nodes: 4 cluster:  True
  288/10000 [...........................

...

 4682/10000 [=============>................] - ETA: 16:19 - reward: 1.9752Starting Action:  4.0

State:  s0 action:  4.0 reward:  2 # nodes: 4 cluster:  True
 4683/10000 [=============>................] - ETA: 16:19 - reward: 1.9752Starting Action:  0.0

State:  s0 action:  0.0 reward:  2 # nodes: 4 cluster:  True
Starting Action:  4.0

State:  s0 action:  4.0 reward:  2 # nodes: 4 cluster:  True
 4685/10000 [=============>................] - ETA: 16:18 - reward: 1.9752Starting Action:  2.0

State:  s0 action:  2.0 reward:  2 # nodes: 4 cluster:  True
Starting Action:  4.0

State:  s0 action:  4.0 reward:  2 # nodes: 4 cluster:  True
 4687/10000 [=============>................] - ETA: 16:18 - reward: 1.9753Starting Action:  9.0

State:  s9 action:  9.0 reward:  2 # nodes: 4 cluster:  True
Starting Action:  5.0

State:  s0 action:  5.0 reward:  2 # nodes: 4 cluster:  True
 4689/10000 [=============>................] - ETA: 16:17 - reward: 1.9753Starting Action:  0.0

State:  s0 action: 

 4739/10000 [=============>................] - ETA: 16:01 - reward: 1.9755Starting Action:  4.0

State:  s0 action:  4.0 reward:  2 # nodes: 4 cluster:  True
 4740/10000 [=============>................] - ETA: 16:01 - reward: 1.9755Starting Action:  9.0

State:  s9 action:  9.0 reward:  2 # nodes: 4 cluster:  True
 4741/10000 [=============>................] - ETA: 16:01 - reward: 1.9755Starting Action:  0.0

State:  s0 action:  0.0 reward:  2 # nodes: 4 cluster:  True
 4742/10000 [=============>................] - ETA: 16:00 - reward: 1.9755Starting Action:  4.0

State:  s0 action:  4.0 reward:  2 # nodes: 4 cluster:  True
 4743/10000 [=============>................] - ETA: 16:00 - reward: 1.9755Starting Action:  2.0

State:  s0 action:  2.0 reward:  2 # nodes: 4 cluster:  True
 4744/10000 [=============>................] - ETA: 16:00 - reward: 1.9755Starting Action:  2.0

State:  s0 action:  2.0 reward:  2 # nodes: 4 cluster:  True
 4745/10000 [=============>................] - ETA: 

 4797/10000 [=============>................] - ETA: 15:43 - reward: 1.9758Starting Action:  2.0

State:  s0 action:  2.0 reward:  2 # nodes: 4 cluster:  True
 4798/10000 [=============>................] - ETA: 15:42 - reward: 1.9758Starting Action:  9.0

State:  s9 action:  9.0 reward:  2 # nodes: 4 cluster:  True
 4799/10000 [=============>................] - ETA: 15:42 - reward: 1.9758Starting Action:  5.0

State:  s0 action:  5.0 reward:  2 # nodes: 4 cluster:  True
 4800/10000 [=============>................] - ETA: 15:42 - reward: 1.9758Starting Action:  9.0

State:  s9 action:  9.0 reward:  2 # nodes: 4 cluster:  True
Starting Action:  2.0

State:  s9 action:  2.0 reward:  2 # nodes: 4 cluster:  True
 4802/10000 [=============>................] - ETA: 15:41 - reward: 1.9758Starting Action:  9.0

State:  s9 action:  9.0 reward:  2 # nodes: 4 cluster:  True
Starting Action:  5.0

State:  s0 action:  5.0 reward:  2 # nodes: 4 cluster:  True
 4804/10000 [=============>...............


State:  s9 action:  9.0 reward:  2 # nodes: 4 cluster:  True
 4916/10000 [=============>................] - ETA: 15:06 - reward: 1.9764Starting Action:  5.0

State:  s0 action:  5.0 reward:  2 # nodes: 4 cluster:  True
 4917/10000 [=============>................] - ETA: 15:06 - reward: 1.9764Starting Action:  9.0

State:  s9 action:  9.0 reward:  2 # nodes: 4 cluster:  True
Starting Action:  9.0

State:  s9 action:  9.0 reward:  2 # nodes: 4 cluster:  True
 4919/10000 [=============>................] - ETA: 15:05 - reward: 1.9764Starting Action:  0.0

State:  s0 action:  0.0 reward:  2 # nodes: 4 cluster:  True
 4920/10000 [=============>................] - ETA: 15:05 - reward: 1.9764Starting Action:  0.0

State:  s0 action:  0.0 reward:  2 # nodes: 4 cluster:  True
 4921/10000 [=============>................] - ETA: 15:04 - reward: 1.9764Starting Action:  2.0

State:  s0 action:  2.0 reward:  2 # nodes: 4 cluster:  True
 4922/10000 [=============>................] - ETA: 15:04 - rewa

 4971/10000 [=============>................] - ETA: 14:50 - reward: 1.9767Starting Action:  5.0

State:  s0 action:  5.0 reward:  2 # nodes: 4 cluster:  True
 4972/10000 [=============>................] - ETA: 14:49 - reward: 1.9767Starting Action:  5.0

State:  s0 action:  5.0 reward:  2 # nodes: 4 cluster:  True
 4973/10000 [=============>................] - ETA: 14:49 - reward: 1.9767Starting Action:  0.0

State:  s0 action:  0.0 reward:  2 # nodes: 4 cluster:  True
 4974/10000 [=============>................] - ETA: 14:49 - reward: 1.9767Starting Action:  0.0

State:  s0 action:  0.0 reward:  2 # nodes: 4 cluster:  True
 4975/10000 [=============>................] - ETA: 14:49 - reward: 1.9767Starting Action:  0.0

State:  s0 action:  0.0 reward:  2 # nodes: 4 cluster:  True
 4976/10000 [=============>................] - ETA: 14:48 - reward: 1.9767Starting Action:  2.0

State:  s0 action:  2.0 reward:  2 # nodes: 4 cluster:  True
 4977/10000 [=============>................] - ETA: 

In [21]:
dqn.save_weights(weights_filename, overwrite=True)


In [ ]:
dqn.load_weights(weights_filename)

In [ ]:
dqn.test(env, nb_episodes=10,  visualize=False)

In [ ]:
print('completed!')